In [1]:
import os
import pandas as pd

# 1. Đường dẫn tới file vocab.txt (chỉnh lại cho đúng bên bạn)
vocab_path = "../../chunkformer-large-vie/vocab.txt"

if not os.path.exists(vocab_path):
    raise FileNotFoundError(f"Không tìm thấy file vocab.txt tại {vocab_path}. Vui lòng kiểm tra lại đường dẫn.")

# 2. Danh sách 29 ký tự cơ bản tiếng Việt
vietnamese_chars = list("aăâbcdđeêghiklmnoôơpqrstuưvy")

# 3. Load toàn bộ token từ vocab.txt
tokens = []
with open(vocab_path, encoding="utf-8") as f:
    for line in f:
        token = line.strip().split()[0]
        tokens.append(token)

# 4. Kiểm tra sự xuất hiện của mỗi ký tự
results = []
for char in vietnamese_chars:
    # Một token có thể là ký tự đơn (e.g. 'a') hoặc subword (e.g. '▁a')
    found = any(tok == char or tok == f"▁{char}" for tok in tokens)
    results.append({"character": char, "found_in_vocab": found})

# 5. Hiển thị kết quả
df = pd.DataFrame(results)
print(df.to_string(index=False))


character  found_in_vocab
        a            True
        ă            True
        â            True
        b            True
        c            True
        d            True
        đ            True
        e            True
        ê            True
        g            True
        h            True
        i            True
        k            True
        l            True
        m            True
        n            True
        o            True
        ô            True
        ơ            True
        p            True
        q            True
        r            True
        s            True
        t            True
        u            True
        ư            True
        v            True
        y            True


In [6]:
# 1. Load vocab.txt
vocab_path = "../../chunkformer-large-vie/vocab.txt"  # chỉnh lại đường dẫn nếu cần
vocab = []
with open(vocab_path, encoding="utf-8") as f:
    for line in f:
        token = line.strip().split()[0]
        vocab.append(token)

# 2. Build mapping token -> ID and sorted vocab for greedy
token2id = {token: idx for idx, token in enumerate(vocab)}
vocab_sorted = sorted(vocab, key=len, reverse=True)

# 3. Greedy longest-match tokenization function
def greedy_tokenize(text: str):
    s = "▁" + text.strip().replace(" ", "▁")
    idx = 0
    L = len(s)
    ids = []
    while idx < L:
        for token in vocab_sorted:
            if s.startswith(token, idx):
                ids.append(token2id[token])
                idx += len(token)
                break
        else:
            unk_id = token2id.get("<unk>")
            if unk_id is not None:
                ids.append(unk_id)
            idx += 1
    return ids

# 4. Example usage
sample_text = "hôm nay trời trói trang quá"
# "hôm nay trời chói chang quá"
# "hum nay trời chói chang quớ"
# "nửa vòng trái đất hơn bảy năm"
token_ids = greedy_tokenize(sample_text)

# 5. Map back IDs -> tokens and print
tokens = [vocab[i] for i in token_ids]
print("Text:      ", sample_text)
print("Token IDs: ", token_ids)
print("Tokens:    ", tokens)

# Or, print one per line:
for idx, tok in zip(token_ids, tokens):
    print(f"{idx}: {tok}")


Text:       hôm nay trời trói trang quá
Token IDs:  [2967, 4136, 5954, 5876, 5818, 4918]
Tokens:     ['▁hôm', '▁nay', '▁trời', '▁trói', '▁trang', '▁quá']
2967: ▁hôm
4136: ▁nay
5954: ▁trời
5876: ▁trói
5818: ▁trang
4918: ▁quá


In [7]:
# 1. Load vocab.txt
vocab_path = "../../chunkformer-large-vie/vocab.txt"
vocab = []
with open(vocab_path, encoding="utf-8") as f:
    for line in f:
        token = line.strip().split()[0]
        vocab.append(token)

# 2. Build mapping and sorted vocab
token2id = {token: idx for idx, token in enumerate(vocab)}
vocab_sorted = sorted(vocab, key=len, reverse=True)

# 3. Greedy longest-match tokenizer
def greedy_tokenize(text: str):
    s = "▁" + text.strip().replace(" ", "▁")
    idx = 0
    L = len(s)
    ids = []
    while idx < L:
        for token in vocab_sorted:
            if s.startswith(token, idx):
                ids.append(token2id[token])
                idx += len(token)
                break
        else:
            # fallback: map char to itself if in vocab, else <unk>
            ch = s[idx]
            if ch in token2id:
                ids.append(token2id[ch])
            else:
                unk_id = token2id.get("<unk>")
                if unk_id is not None:
                    ids.append(unk_id)
            idx += 1
    return ids

# 4. Test cases
test_texts = [
    "xử lý dữ liệu",
    "quốc hội",
    "self-service",
    "đặc biệt",
    "học máy nâng cao",
]

for text in test_texts:
    ids = greedy_tokenize(text)
    toks = [vocab[i] for i in ids]
    print(f"\nText:      {text}")
    print(f"Token IDs: {ids}")
    print(f"Tokens:    {toks}")



Text:      xử lý dữ liệu
Token IDs: [6591, 3636, 2433, 3517]
Tokens:    ['▁xử', '▁lý', '▁dữ', '▁liệu']

Text:      quốc hội
Token IDs: [4966, 3059]
Tokens:    ['▁quốc', '▁hội']

Text:      self-service
Token IDs: [5216, 1, 740, 854, 159]
Tokens:    ['▁self', '<unk>', 'ser', 'vi', 'ce']

Text:      đặc biệt
Token IDs: [6844, 1484]
Tokens:    ['▁đặc', '▁biệt']

Text:      học máy nâng cao
Token IDs: [3037, 3958, 4541, 1755]
Tokens:    ['▁học', '▁máy', '▁nâng', '▁cao']
